In [1]:
'''
selenium 셀레니움 설치+크롬 드라이버 설치
https://chancoding.tistory.com/136

참고
https://dodonam.tistory.com/228

현재 아이디와 비밀번호를 넣지 않아 오류가 발생했음(개인정보라 혹시 몰라서 일부러 오류 발생시킴)

'''

import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import re
import pandas as pd

#웹드라이버 여는 거
driver = webdriver.Chrome()

#html태그를 지우는거
def remove_html_tags(data):
    p = re.compile(r'<.*?>') #정규표현식 
    #.*? : Non-Greedy / 최소 반복 수행
    #< ~~~~ > 이렇게 끝나는 듯 
    #https://whatisthenext.tistory.com/116 참고
    return p.sub(' ',str(data))
    

#원하는 표만 태그 지우고 값들만 가져오는 함수
def get_crawl(URL):
    response = driver.get(URL)
    html = driver.page_source
    soup7 = BeautifulSoup(html,'html.parser')
    ex_id_divs = soup7.find('table',{'id':'gdasComplSubjDscListTbl'}) #과목들 정리된 표만 가져옴
    tds = ex_id_divs.find_all('td') # td 태그만
    crawl_data = remove_html_tags(tds) #태그 지우기
    remove_space = crawl_data.replace(" ","") #공백 지우기
    remove_space = remove_space.strip("[""]") #맨 앞과 맨 뒤에 있던 []지우기
    remove_space = remove_space.split(',') #문자열을 리스트로
    return remove_space #표에 있는 값들 반환
    
    
    

driver.implicitly_wait(3)
driver.get('https://webinfo.dankook.ac.kr/member/logon.do') #로그인 홈페이지
login_x_path='//*[@id="logonForm"]/div[3]/button' #로그인 버튼 위치
driver.find_element_by_name('username').send_keys('<아이디/>') #웹정보 아이디
driver.find_element_by_name('password').send_keys('<비밀번호/>') #웹정보 비밀번호
driver.find_element_by_xpath(login_x_path).click() # 로그인 버튼 클릭

#확인하고 싶은 위치
target_url = "https://webinfo.dankook.ac.kr/tiac/univ/grad/gasm/views/findGrdtObjrDscGeneration.do"
lists = get_crawl(target_url)


C:\Users\psy33\AppData\Local\Temp/ipykernel_6300/3879185487.py:47: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_name('username').send_keys('<아이디/>') #웹정보 아이디
C:\Users\psy33\AppData\Local\Temp/ipykernel_6300/3879185487.py:48: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_name('password').send_keys('<비밀번호/>') #웹정보 비밀번호
C:\Users\psy33\AppData\Local\Temp/ipykernel_6300/3879185487.py:49: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath(login_x_path).click() # 로그인 버튼 클릭


AttributeError: 'NoneType' object has no attribute 'find_all'

In [3]:
#데이터프레임 생성

#기본적인 데이터프레임 생성
df = pd.DataFrame([lists[0:9]],index=list(range(1,int(len(lists)/10)+1)))

#반복문으로 데이터프레임화
for i in range(10,len(lists),10):
    df.loc[int(i/10)+1] = lists[i:i+9]
#colums 지정
df.columns =['전공구분','이수조건','이수영역','교육과정영역','코드','교과목명','이수년도','이수학기','학점']
df['학점'] = pd.to_numeric(df['학점']) #문자열 정수로 변경

In [2]:
#데이터 프레임 확인
df

NameError: name 'df' is not defined

In [5]:
#교양, 전공소양, 심화전공, 기타 데이터프레임 생성
#추후 필요하면 출력해보기

ge_df = df[df['이수조건'].str.contains('교양과정')] #교양 데이터프레임
sw_ge_df = df[df['이수조건'].str.contains('전공소양')] #전공소양 데이터프레임
major_df = df[df['이수조건'].str.contains('심화전공')] #전공 데이터프레임
etc_df = df[~df['이수조건'].str.contains('|'.join(['전공','교양']))] #전공,교양 단어가 들어간거 제외

In [14]:
#학점 수 더한 데이터프레임 생성
df_list = [df['학점'].sum(),
ge_df['학점'].sum(),
sw_ge_df['학점'].sum(),
major_df['학점'].sum(),
etc_df['학점'].sum()]

sum_df = pd.DataFrame([[140,20,20,79,0],df_list],columns=['전체 학점','교양 학점','전공소양 학점','전공 학점','기타 학점'],index=['필요 최소 학점','현재 학점'])

sum_df.loc['남은 학점'] = -sum_df.diff().iloc[1] #필요 학점 - 현재 학점 값


In [3]:
#학점 합 확인
sum_df

NameError: name 'sum_df' is not defined